In [6]:
import cv2
import numpy as np

# 读取视频
cap = cv2.VideoCapture(r'D:\deepsea\data\video\demo.mp4')

# 获取视频的基本信息
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 输出视频设置
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(r'D:\deepsea\data\video\output_stabilized.mp4', fourcc, fps, (width, height))

# 前一帧，用于计算帧间运动
ret, prev_frame = cap.read()
if not ret:
    raise ValueError("Failed to read the first frame from the video.")
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# 用于存储累积变换矩阵的变量
transforms = np.zeros((n_frames - 1, 3), np.float32)

# Step 1: 计算每对连续帧之间的相对运动
for i in range(n_frames - 2):
    # 读取当前帧
    ret, curr_frame = cap.read()
    if not ret:
        break
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

    # 使用GFTT角点检测来找到前一帧的特征点
    prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=200, qualityLevel=0.01, minDistance=30, blockSize=3)

    # 使用光流法追踪这些特征点
    curr_pts, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None)

    # 过滤有效的特征点
    valid_prev_pts = prev_pts[status == 1]
    valid_curr_pts = curr_pts[status == 1]

    # 估计相对变换（使用仿射变换）
    matrix, _ = cv2.estimateAffinePartial2D(valid_prev_pts, valid_curr_pts)

    # 从变换矩阵中提取出平移、缩放和旋转信息
    dx = matrix[0, 2]
    dy = matrix[1, 2]
    da = np.arctan2(matrix[1, 0], matrix[0, 0])

    # 存储变换参数
    transforms[i] = [dx, dy, da]

    # 更新前一帧
    prev_gray = curr_gray

# Step 2: 累积变换的平滑处理
trajectory = np.cumsum(transforms, axis=0)
smooth_trajectory = cv2.GaussianBlur(trajectory, (15, 15), 5)

# 计算平滑后的变换量
diff = smooth_trajectory - trajectory
transforms_smooth = transforms + diff

# Step 3: 应用平滑后的变换到视频帧
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
for i in range(n_frames - 2):
    ret, frame = cap.read()
    if not ret:
        break

    # 获取平滑后的变换参数
    dx = transforms_smooth[i, 0]
    dy = transforms_smooth[i, 1]
    da = transforms_smooth[i, 2]

    # 构建平滑后的仿射变换矩阵
    matrix = np.array([[np.cos(da), -np.sin(da), dx],
                       [np.sin(da), np.cos(da), dy]])

    # 应用仿射变换到当前帧
    frame_stabilized = cv2.warpAffine(frame, matrix, (width, height))

    # 将稳定后的帧写入输出视频
    out.write(frame_stabilized)

    # 显示处理过程（可选）
    cv2.imshow("Stabilized Frame", frame_stabilized)
    cv2.waitKey(10)

# 释放视频捕获和写入对象
cap.release()
out.release()
cv2.destroyAllWindows()


In [11]:
import cv2
import numpy as np

# 读取视频
cap = cv2.VideoCapture(r'D:\deepsea\data\video\demo.mp4')

# 获取视频的基本信息
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# 输出视频设置
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(r'D:\deepsea\data\video\output_stabilized.mp4', fourcc, fps, (width, height))

# 前一帧，用于计算帧间运动
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# 用于存储累积平移量的变量
transforms = np.zeros((n_frames - 1, 2), np.float32)

# Step 1: 计算每对连续帧之间的相对平移运动
for i in range(n_frames - 2):
    # 读取当前帧
    ret, curr_frame = cap.read()
    if not ret:
        break
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

    # 使用光流法计算帧间特征点运动
    prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=200, qualityLevel=0.01, minDistance=30, blockSize=3)
    curr_pts, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_pts, None)

    # 过滤有效的特征点
    valid_prev_pts = prev_pts[status == 1]
    valid_curr_pts = curr_pts[status == 1]

    # 计算平移（取平均的位移）
    dx = np.mean(valid_curr_pts[:, 0] - valid_prev_pts[:, 0])
    dy = np.mean(valid_curr_pts[:, 1] - valid_prev_pts[:, 1])

    # 存储变换参数
    transforms[i] = [dx, dy]

    # 更新前一帧
    prev_gray = curr_gray

# Step 2: 累积平移量的平滑处理
trajectory = np.cumsum(transforms, axis=0)
smooth_trajectory = cv2.GaussianBlur(trajectory, (15, 15), 5)

# 计算平滑后的变换量
diff = smooth_trajectory - trajectory
transforms_smooth = transforms + diff

# Step 3: 应用平滑后的变换到视频帧
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
for i in range(n_frames - 2):
    ret, frame = cap.read()
    if not ret:
        break

    # 获取平滑后的平移参数
    dx = transforms_smooth[i, 0]
    dy = transforms_smooth[i, 1]

    # 扩大画布以便处理边界
    border_size = 50  # 这个值可以根据需要调整
    frame_expanded = cv2.copyMakeBorder(frame, border_size, border_size, border_size, border_size, cv2.BORDER_REPLICATE)

    # 构建仿射变换矩阵，仅包含平移
    transform_matrix = np.array([[1, 0, dx],
                                 [0, 1, dy]], dtype=np.float32)

    # 应用仿射变换到扩大后的帧
    frame_stabilized = cv2.warpAffine(frame_expanded, transform_matrix, (width + 2 * border_size, height + 2 * border_size))

    # 裁剪回原来的尺寸
    x_start = border_size
    y_start = border_size
    frame_cropped = frame_stabilized[y_start:y_start + height, x_start:x_start + width]

    # 将稳定后的帧写入输出视频
    out.write(frame_cropped)

    # 显示处理过程（可选）
    cv2.imshow("Stabilized Frame", frame_cropped)
    cv2.waitKey(10)

# 释放视频捕获和写入对象
cap.release()
out.release()
cv2.destroyAllWindows()

In [12]:
import cv2
import numpy as np

# 读取视频
cap = cv2.VideoCapture(r'D:\deepsea\data\video\demo.mp4')

# 输出视频设置
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(r'D:\deepsea\data\video\output_stabilized.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS),
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Step 1: 转换为灰度图像
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Step 2: 高斯模糊，减少噪声
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)

    # Step 3: 使用霍夫圆检测找到气泡
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=20,
                               param1=50, param2=30, minRadius=5, maxRadius=50)

    # Step 4: 在检测到的气泡上绘制圆圈
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            # 绘制气泡的圆
            cv2.circle(frame, (x, y), r, (0, 255, 0), 4)
            # 绘制圆心
            cv2.circle(frame, (x, y), 2, (0, 0, 255), 3)

    # Step 5: 将标记后的帧保存到输出视频中
    out.write(frame)

    # 显示处理过程（可选）
    cv2.imshow("Bubbles Detected", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 释放视频捕获和写入对象
cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import os

# 视频文件路径
video_path = r'D:\deepsea\data\video\bubble1.mp4'

# 抽取的帧保存路径
output_folder = r'D:\deepsea\data\bubble1_frames'

# 创建输出文件夹（如果不存在）
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 打开视频文件
cap = cv2.VideoCapture(video_path)

# 检查视频是否成功打开
if not cap.isOpened():
    raise ValueError(f"Cannot open video file: {video_path}")

# 获取视频的帧率
fps = int(cap.get(cv2.CAP_PROP_FPS))

# 每隔多少帧保存一帧（例如每秒抽取1帧）
frame_interval = fps

# 当前帧计数器
frame_count = 0

# 抽取帧并保存
while True:
    ret, frame = cap.read()

    # 如果无法读取到帧，结束循环
    if not ret:
        break

    # 如果当前帧是目标帧之一（根据帧间隔）
    if frame_count % frame_interval == 0:
        # 生成保存的图片文件名
        frame_filename = os.path.join(output_folder, f'frame_{frame_count:04d}.jpg')
        # 保存该帧
        cv2.imwrite(frame_filename, frame)

    # 递增帧计数器
    frame_count += 1

# 释放视频文件
cap.release()

print(f"Frames have been saved to the folder: {output_folder}")


Frames have been saved to the folder: D:\deepsea\data\frames


In [ ]:
import torch

from segment_anything import sam_model_registry as smr
from segment_anything import SamAutomaticMaskGenerator as SamAMG
from segment_anything import SamPredictor as SamP

# 加载预训练模型
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_TYPE = "vit_b"

sam = smr[MODEL_TYPE](checkpoint=r'D:\deepocean\deepsea\models\SAM\segment-anything\sam_vit_b_01ec64.pth').to(device=DEVICE)


In [ ]:
import os
import cv2
import torch
from pathlib import Path

# 加载 YOLOv5 模型
model = torch.hub.load('D:\deepocean\deepsea\models\yolov5-master', 'custom', path='yolov5s.pt', source='local')  # 使用预训练模型

# 输入和输出文件夹路径
input_folder = 'frames'
output_folder = 'data/frames_result'

# 创建输出文件夹（如果不存在）
Path(output_folder).mkdir(parents=True, exist_ok=True)

# 处理每张图片
for img_name in os.listdir(input_folder):
    img_path = os.path.join(input_folder, img_name)
    img = cv2.imread(img_path)

    # 使用 YOLO 模型进行预测
    results = model(img)

    # 在图片上绘制检测结果
    results.render()

    # 保存结果图片
    output_path = os.path.join(output_folder, img_name)
    cv2.imwrite(output_path, img)

print("所有图片处理完成，结果保存在 'data/frames_result/' 文件夹中。")

In [1]:
import os
import cv2
import torch
from pathlib import Path

# 加载训练好的 YOLOv5 模型
model = torch.hub.load(r'D:\deepocean\deepsea\models\yolov5-master', 'custom', path=r'D:\deepocean\deepsea\my_project\my_experiment7\weights\best.pt', source='local')

# 输入和输出文件夹路径
input_folder = r'D:\deepocean\deepsea\data\frames'
output_folder = r'D:\deepocean\deepsea\data\frames_result'

# 创建输出文件夹（如果不存在）
Path(output_folder).mkdir(parents=True, exist_ok=True)

# 处理每张图片
for img_name in os.listdir(input_folder):
    img_path = os.path.join(input_folder, img_name)
    img = cv2.imread(img_path)

    # 使用 YOLO 模型进行预测
    results = model(img)

    # 在图片上绘制检测结果
    results.render()

    # 保存结果图片
    output_path = os.path.join(output_folder, img_name)
    cv2.imwrite(output_path, img)

print("所有图片处理完成，结果保存在 'data/frames_result/' 文件夹中。")

YOLOv5  2024-10-27 Python-3.10.15 torch-2.5.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
D:\deepocean\deepsea\models\yolov5-master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
D:\deepocean\deepsea\models\yolov5-master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
D:\deepocean\deepsea\models\yolov5-master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
D:\deepocean\deepsea\models\yolov5-master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. 

所有图片处理完成，结果保存在 'data/frames_result/' 文件夹中。


In [4]:
import cv2
import os

# 视频文件路径
video_path = r'D:\deepocean\deepsea\data\video\bubble1.mp4'

# 抽取的帧保存路径
output_folder = r'D:\deepocean\deepsea\data\bubble1_frames'

# 创建输出文件夹（如果不存在）
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 打开视频文件
cap = cv2.VideoCapture(video_path)

# 检查视频是否成功打开
if not cap.isOpened():
    raise ValueError(f"Cannot open video file: {video_path}")

# 获取视频的帧率
fps = int(cap.get(cv2.CAP_PROP_FPS))

# 每隔多少帧保存一帧（例如每秒抽取1帧）
frame_interval = fps

# 当前帧计数器
frame_count = 0

# 抽取帧并保存
while True:
    ret, frame = cap.read()

    # 如果无法读取到帧，结束循环
    if not ret:
        break

    # 如果当前帧是目标帧之一（根据帧间隔）
    if frame_count % frame_interval == 0:
        # 生成保存的图片文件名
        frame_filename = os.path.join(output_folder, f'frame_{frame_count:04d}.jpg')
        # 保存该帧
        cv2.imwrite(frame_filename, frame)

    # 递增帧计数器
    frame_count += 1

# 释放视频文件
cap.release()

print(f"Frames have been saved to the folder: {output_folder}")


Frames have been saved to the folder: D:\deepocean\deepsea\data\bubble1_frames


In [7]:
import os
import cv2
import torch
from pathlib import Path

# 加载训练好的 YOLOv5 模型
model = torch.hub.load(r'D:\deepocean\deepsea\models\yolov5-master', 'custom', path=r'D:\deepocean\deepsea\my_project\my_experiment7\weights\best.pt', source='local')

# 输入和输出文件夹路径
input_folder = r'D:\deepocean\deepsea\data\bubble1_frames'
output_folder = r'D:\deepocean\deepsea\data\bubble1_frames_result'

# 创建输出文件夹（如果不存在）
Path(output_folder).mkdir(parents=True, exist_ok=True)

# 处理每张图片
for img_name in os.listdir(input_folder):
    img_path = os.path.join(input_folder, img_name)
    img = cv2.imread(img_path)

    # 使用 YOLO 模型进行预测
    results = model(img)

    # 在图片上绘制检测结果
    results.render()

    # 保存结果图片
    output_path = os.path.join(output_folder, img_name)
    cv2.imwrite(output_path, img)

print("所有图片处理完成，结果保存在 'data/frames_result/' 文件夹中。")

YOLOv5  2024-10-27 Python-3.10.15 torch-2.5.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
D:\deepocean\deepsea\models\yolov5-master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
D:\deepocean\deepsea\models\yolov5-master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
D:\deepocean\deepsea\models\yolov5-master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
D:\deepocean\deepsea\models\yolov5-master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. 

所有图片处理完成，结果保存在 'data/frames_result/' 文件夹中。


D:\deepocean\deepsea\models\yolov5-master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


In [8]:
import os
import cv2
import torch
import sys

# 将SAM模型文件夹添加到系统路径
sys.path.append('D:/deepocean/deepsea/sam')

from segment_anything import sam_model_registry, SamPredictor

# 加载SAM模型
sam_checkpoint = r"D:\deepocean\deepsea\models\SAM\segment-anything\sam_vit_b_01ec64.pth"
model_type = "vit_b"
device = "cuda" if torch.cuda.is_available() else "cpu"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

# 图片文件夹路径
image_folder = 'D:/deepocean/deepsea/data/bubble1_frames'

# 遍历文件夹中的所有图片
for filename in os.listdir(image_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(image_folder, filename)
        image = cv2.imread(image_path)
        
        # 使用SAM模型进行气泡检测
        predictor.set_image(image)
        masks, _, _ = predictor.predict()
        
        # 在图片上绘制检测到的气泡
        for mask in masks:
            contours, _ = cv2.findContours(mask.astype('uint8'), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # 保存带有检测结果的图片
        result_path = os.path.join(image_folder, 'results', filename)
        os.makedirs(os.path.dirname(result_path), exist_ok=True)
        cv2.imwrite(result_path, image)

print("气泡检测完成。")

气泡检测完成。


In [12]:
import os
import cv2
import json

# 图片文件夹路径
image_folder = 'D:/deepocean/deepsea/data/bubble1_frames'
annotations = {}

# 鼠标回调函数
def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, img, img_copy, rects

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
        print(f"Mouse down: ({ix}, {iy})")

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img = img_copy.copy()
            cv2.rectangle(img, (ix, iy), (x, y), (0, 255, 0), 2)
            print(f"Mouse move: ({x}, {y})")

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        rects.append((ix, iy, x, y))
        cv2.rectangle(img, (ix, iy), (x, y), (0, 255, 0), 2)
        print(f"Mouse up: ({x}, {y})")

# 遍历文件夹中的所有图片
for filename in os.listdir(image_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(image_folder, filename)
        img = cv2.imread(image_path)
        img_copy = img.copy()
        rects = []

        cv2.namedWindow('image')
        cv2.setMouseCallback('image', draw_rectangle)

        while True:
            cv2.imshow('image', img)
            k = cv2.waitKey(1) & 0xFF
            if k == ord('q'):  # 按 'q' 键退出当前图片标注
                break
            elif k == ord('s'):  # 按 's' 键保存当前图片的标注并继续下一张图片
                if filename not in annotations:
                    annotations[filename] = []
                annotations[filename].extend(rects)
                break

        cv2.destroyAllWindows()

# 保存标注结果为JSON文件
with open('D:/deepocean/deepsea/annotations.json', 'w') as f:
    json.dump(annotations, f)

print("标注完成并保存。")

Mouse down: (1106, 143)
Mouse move: (1107, 143)
Mouse move: (1107, 143)
Mouse move: (1109, 143)
Mouse move: (1112, 144)
Mouse move: (1117, 144)
Mouse move: (1120, 145)
Mouse move: (1122, 145)
Mouse move: (1125, 146)
Mouse move: (1128, 147)
Mouse move: (1130, 147)
Mouse move: (1133, 147)
Mouse move: (1135, 147)
Mouse move: (1137, 148)
Mouse move: (1137, 148)
Mouse move: (1137, 149)
Mouse move: (1137, 149)
Mouse move: (1137, 151)
Mouse move: (1137, 151)
Mouse move: (1136, 152)
Mouse move: (1136, 153)
Mouse move: (1136, 154)
Mouse move: (1135, 154)
Mouse move: (1135, 155)
Mouse move: (1134, 155)
Mouse move: (1134, 156)
Mouse move: (1134, 156)
Mouse move: (1133, 157)
Mouse move: (1133, 159)
Mouse move: (1132, 159)
Mouse move: (1131, 161)
Mouse move: (1131, 161)
Mouse move: (1130, 162)
Mouse move: (1130, 163)
Mouse move: (1130, 164)
Mouse move: (1129, 164)
Mouse move: (1129, 164)
Mouse move: (1128, 164)
Mouse move: (1127, 164)
Mouse move: (1125, 164)
Mouse move: (1125, 164)
Mouse move: (112